## Nuova versione di AccoppiamentoDati.ipynb -> in questa versione molte funzioni sono importate dal file utilstre.py

In [1]:
# --- Config ---
import sys, os, pickle, numpy as np
# Verifica che la cartella lib esista
lib_path = 'lib'  # Percorso della cartella lib/
assert os.path.isdir(lib_path), f"Percorso lib non trovato: {lib_path}"

# Aggiungi lib/ al path se non è già presente
if lib_path not in sys.path:
    sys.path.append(lib_path)


from lib.utilstre import (
    match_patient_txt_files,
    make_npz_from_txt_files,
    load_data_by_patient,
    save_split_npz,
)

# Cartelle con i .txt per i due task (A e B)

FOLDER_A= '../../HCP/RestingStateLR_dataset/'
FOLDER_B = '../../HCP/LanguageTaskLR_dataset/' 
#FOLDER_B = '../../HCP/MemoryTaskLR_dataset/'
 

# Dove salvare i .npz intermedi (uno per task) e lo split finale
NPZ_A = "TaskA.npz"      # es. "RestingState.npz"
NPZ_B = "TaskB.npz"      # es. "WorkingMemory.npz"
OUT_SPLIT = "split_caseX_A_vs_B.npz"  # verrà aggiornato più sotto

# Colonne da scartare all'inizio (le 6 di riferimento paziente)
KEEP_FIRST_K = 6

# come gestire lunghezze T diverse intra-task e tra task: "min" oppure "pad"
ALIGN_T_INTRA  = "min"   # nella costruzione dei singoli .npz
ALIGN_T_ACROSS = "min"   # quando carichiamo A e B insieme

# split e seed
CASE = 1            # <-- scegli 1, 2 o 3
# case=1: Train su A (subset pazienti), Val/Test su B sugli *stessi pazienti*.
# case=2: Train su A (subset pazienti), Val/Test su A su *pazienti diversi*.
# case=3: Train su A (subset pazienti), Val/Test su B su *pazienti diversi*.

TEST_SIZE = 0.2
VAL_SIZE  = 0.2
SEED = 42


In [2]:
filesA, filesB, ids = match_patient_txt_files(FOLDER_A, FOLDER_B)
print(f"Pazienti in comune: {len(ids)}")
print("Esempio ID:", ids[:5])

a_npz_path, a_shape, a_ids = make_npz_from_txt_files(
    filesA, NPZ_A, keep_first_k=KEEP_FIRST_K, align_T=ALIGN_T_INTRA
)
b_npz_path, b_shape, b_ids = make_npz_from_txt_files(
    filesB, NPZ_B, keep_first_k=KEEP_FIRST_K, align_T=ALIGN_T_INTRA
)


print("A ->", a_npz_path, "shape:", a_shape)
print("B ->", b_npz_path, "shape:", b_shape)
assert len(a_ids) == len(b_ids) == len(ids)


Pazienti in comune: 1042
Esempio ID: ['100206', '100307', '100408', '100610', '101006']
A -> TaskA.npz shape: (1042, 922, 6)
B -> TaskB.npz shape: (1042, 288, 6)


In [3]:
# Nota: enforce_same_order True perché i due .npz ora hanno lo stesso ordine di patient_ids
scalers, Xtr, Xval, Xte, train_ids, test_ids = load_data_by_patient(
    dataA_path=NPZ_A,
    dataB_path=NPZ_B,
    case=CASE,
    test_size=TEST_SIZE,
    val_size=VAL_SIZE,
    random_state=SEED,
    enforce_same_order=True,
    align_T_mode=ALIGN_T_ACROSS,
)

print("TRAIN:", Xtr.shape, "VAL:", Xval.shape, "TEST:", Xte.shape)
print("N train_ids:", len(train_ids), "N test_ids:", len(test_ids))


TRAIN: (833, 288, 6) VAL: (416, 288, 6) TEST: (417, 288, 6)
N train_ids: 833 N test_ids: 833


In [5]:
# Costruisci un nome file significativo
OUT_SPLIT = f"split_case{CASE}_A_{os.path.splitext(NPZ_A)[0]}_B_{os.path.splitext(NPZ_B)[0]}.npz"

from utilstre import save_split_npz
path_out = save_split_npz(
    out_path=OUT_SPLIT,
    X_train=Xtr, X_val=Xval, X_test=Xte,
    train_ids=train_ids, test_ids=test_ids,
    scalers=scalers,   # opzionale ma utile
)
print("Salvato:", path_out)


Salvato: split_case1_A_TaskA_B_TaskB.npz
